## 1. ¿Cuantos alumnos diferentes hay en la edición de septiembre?

In [ ]:
cursor.execute('''
SELECT DISTINCT (COUNT(a.id))
FROM Bootcamps b
INNER JOIN Promociones p ON b.promocion_id = p.id
INNER JOIN Alumnos a ON b.id = a.bootcamp_id
WHERE p.promocion = 'Septiembre'
''')

## 2. ¿Que campus ha acogido más alumnos?

In [ ]:

cursor.execute('''
SELECT c.campus,count(a.id)
FROM campus c
LEFT JOIN Bootcamps b ON c.id = b.campus_id
LEFT JOIN Alumnos a ON b.id = a.bootcamp_id
GROUP BY c.campus
''')

## 3. Alumnos con al menos un "No Apto"

In [ ]:
cursor.execute('''
WITH alumnos_con_no_apto AS (    -- inicia el cte (common table expression), con el nombre de la tabla temporal
    SELECT DISTINCT alumno_id  --selecciona sin duplicados
    FROM calificaciones
    WHERE calificacion = 'No Apto'
)
SELECT
    a.id,
    a.nombre,
    a.email,
    a.bootcamp_id,
    a.aula
FROM alumnos a
INNER JOIN alumnos_con_no_apto na ON a.id = na.alumno_id -- usamos la cte (tabla temporal), con alias "na", el ON es la condicion de union
ORDER BY a.nombre;
''')

## 4. Alumno con la suma de proyectos que han sido evaluados

In [ ]:
cursor.execute('''
SELECT a.nombre, count(ca.id) AS suma_entregas
FROM alumnos a
LEFT JOIN calificaciones ca ON a.id = ca.alumno_id
GROUP BY a.nombre
ORDER BY suma_entregas DESC;
''')

## 5. % de aprobados y suspensos por campus


In [ ]:
cursor.execute('''
SELECT
    c.campus,
    ROUND(100.0 * SUM(CASE WHEN LOWER(ca.calificacion) = 'apto' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_apto,
    ROUND(100.0 * SUM(CASE WHEN LOWER(ca.calificacion) = 'no apto' THEN 1 ELSE 0 END) / COUNT(*), 2) AS porcentaje_no_apto
FROM Alumnos a
JOIN Bootcamps b ON a.bootcamp_id = b.id
LEFT JOIN Calificaciones ca ON a.id = ca.alumno_id
LEFT JOIN Campus c ON b.campus_id = c.id
GROUP BY c.campus;
''')